In [1]:
# Prerequisites 

from pathlib import Path
import pandas as pd
from ordered_set import OrderedSet
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
import json
from yaml.loader import SafeLoader
import cobra
from cobra.io import load_model
from cobra import Model, Reaction, Metabolite
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model, load_model, load_yaml_model, save_yaml_model
import logging
from cobra.sampling import sample
from cobra.medium import minimal_medium
import ast

# load model

In [2]:
eg_path = Path("/Users/philine/Downloads/yeast-GEM-8.6.2/model")
yml_path = eg_path / "yeast-GEM.yml"
model = load_yaml_model(str(yml_path.resolve()))
model

Restricted license - for non-production use only - expires 2024-10-28


Name,None
Memory address,7feb92f6ff10
Number of metabolites,2744
Number of reactions,4063
Number of genes,1160
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


# prep model

In [3]:
# create function that prepares the model, set solver, objective function and remove original carbon source
def prep_model(model):
    
    model.solver = 'glpk'
    model.objective = 'r_4041'
    
    medium = model.medium
    medium['r_1714'] = 0.0
    model.medium = medium
    
    return model

model = prep_model(model)

# load mapped id dict

In [4]:
#get csv containing s id and kegg id

sid_kegg = pd.read_csv('map_sid_kegg.csv')
sid_kegg

,S ID,KEGG ID
0,s_0002,C00965
1,s_0007,C11514
2,s_0009,C04411
3,s_0010,C04236
4,s_0013,NaN
...,...,...
849,s_4261,NaN
850,s_4264,C00957
851,s_4265,C15521
852,s_4267,C05122


In [33]:
# create dictionary of sid_kegg data frame

sk_dict = dict(zip(sid_kegg['S ID'], sid_kegg['KEGG ID']))

# load csv

In [6]:
# create df to keep original csv for analysis
ogcsv = pd.read_csv('CSV.csv')
ogcsv['INTERMEDIATES KEGG'] = ogcsv['INTERMEDIATES KEGG'].apply(lambda x: ast.literal_eval(x))
ogcsv['INTERMEDIATE NAMES'] = ogcsv['INTERMEDIATE NAMES'].apply(lambda x: ast.literal_eval(x))
ogcsv = ogcsv.drop(['REACTIONS', 'LENGTH SC.'], axis=1)
ogcsv

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES
0,5,"[C11505, C02001, C01471, C05608, C06561, C00509]","[Propene, 2-Propen-1-ol, Propenal, p-Coumarald..."
1,5,"[C11505, C02001, C05855, C01600, C04608, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc..."
2,5,"[C11505, C02001, C05855, C00842, C01460, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc..."
3,5,"[C11505, C02001, C05855, C00842, C01714, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc..."
4,5,"[C11505, C02001, C05855, C00842, C04608, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc..."
...,...,...,...
1722,5,"[C06547, C00511, C04415, C00498, C04608, C01477]","[Ethylene, Acrylate, 4-O-beta-D-Glucosyl-4-hyd..."
1723,5,"[C06547, C00511, C04415, C00498, C01714, C01477]","[Ethylene, Acrylate, 4-O-beta-D-Glucosyl-4-hyd..."
1724,5,"[C06547, C00511, C04415, C04507, C04608, C01477]","[Ethylene, Acrylate, 4-O-beta-D-Glucosyl-4-hyd..."
1725,5,"[C11505, C02001, C01471, C10945, C15525, C05631]","[Propene, 2-Propen-1-ol, Propenal, Caffeyl ald..."


In [7]:
# read csv containing all pathways containing >=1 matches
csv = pd.read_csv('non_perfect.csv')
csv['INTERMEDIATES KEGG'] = csv['INTERMEDIATES KEGG'].apply(lambda x: ast.literal_eval(x))
csv['INTERMEDIATE NAMES'] = csv['INTERMEDIATE NAMES'].apply(lambda x: ast.literal_eval(x))
csv

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES,index
0,5,"[C11505, C20464, C00530, C05608, C06561, C00509]","[Propene, Isochavicol, Quinol, p-Coumaraldehyd...",17
1,5,"[C11505, C02001, C05855, C00096, C01714, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc...",22
2,5,"[C11505, C02001, C05855, C00031, C04608, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc...",23
3,5,"[C11505, C02001, C05855, C00031, C09099, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc...",24
4,5,"[C11505, C02001, C05855, C00096, C01460, C00509]","[Propene, 2-Propen-1-ol, 4-Hydroxycinnamyl alc...",30
...,...,...,...,...
1302,5,"[C06547, C00511, C04415, C00096, C01714, C01477]","[Ethylene, Acrylate, 4-O-beta-D-Glucosyl-4-hyd...",1720
1303,5,"[C06547, C00511, C04415, C00096, C01460, C01477]","[Ethylene, Acrylate, 4-O-beta-D-Glucosyl-4-hyd...",1721
1304,5,"[C06547, C00511, C04415, C00498, C04608, C01477]","[Ethylene, Acrylate, 4-O-beta-D-Glucosyl-4-hyd...",1722
1305,5,"[C06547, C00511, C04415, C00498, C01714, C01477]","[Ethylene, Acrylate, 4-O-beta-D-Glucosyl-4-hyd...",1723


## select df to use

In [8]:
df = csv

# get plastics

In [9]:
# create function that gets all plastics

def get_plastics(df):
    
    csv_pathways = [i for i in df['INTERMEDIATES KEGG']]

    get_plastic = set()

    for i in csv_pathways:
        if i[0] not in get_plastic:
            get_plastic.add(i[0])

    plastics = list(get_plastic)
    return plastics

In [10]:
# apply function to current dataset
plastics = get_plastics(df)
plastics

['C06337', 'C07083', 'C06547', 'C11505', 'C06793']

In [11]:
# create plastic metabolites

## Propene

C11505_e = Metabolite(
    'C11505_e',
    formula = 'C3H6',
    name = 'Propene',
    compartment = 'e')

C11505_c = Metabolite(
    'C11505_c',
    formula = 'C3H6',
    name = 'Propene',
    compartment = 'c')


## Vinyl Chloride

C06793_e = Metabolite(
    'C06793_e',
    formula = 'C2H3Cl',
    name = 'Vinyl Chloride',
    compartment = 'e')

C06793_c = Metabolite(
    'C06793_c',
    formula = 'C2H3Cl',
    name = 'Vinyl Chloride',
    compartment = 'c')


## Ethylene

C06547_e = Metabolite(
    'C06547_e',
    formula = 'C2H4',
    name = 'Ethylene',
    compartment = 'e')

C06547_c = Metabolite(
    'C06547_c',
    formula = 'C2H4',
    name = 'Ethylene',
    compartment = 'c')


## Styrene

C07083_e = Metabolite(
    'C07083_e',
    formula = 'C8H8',
    name = 'Styrene',
    compartment = 'e')

C07083_c = Metabolite(
    'C07083_c',
    formula = 'C8H8',
    name = 'Styrene',
    compartment = 'c')


## Terephthalate 

C06337_e = Metabolite(
    'C06337_e',
    formula = 'C8H6O4',
    name = 'Terephthalate',
    compartment = 'e')

C06337_c = Metabolite(
    'C06337_c',
    formula = 'C8H6O4',
    name = 'Terephthalate',
    compartment = 'c')

In [12]:
# add metabolites to new dictionary containing plastics
plastic_dict = {}

plastic_dict['C11505_c'] = C11505_c
plastic_dict['C06793_c'] = C06793_c
plastic_dict['C06547_c'] = C06547_c
plastic_dict['C07083_c'] = C07083_c
plastic_dict['C06337_c'] = C06337_c

plastic_dict['C11505_e'] = C11505_e
plastic_dict['C06793_e'] = C06793_e
plastic_dict['C06547_e'] = C06547_e
plastic_dict['C07083_e'] = C07083_e
plastic_dict['C06337_e'] = C06337_e

In [13]:
# add plastic metabolites to dictionary

sk_dict['C11505_c'] = C11505_c
sk_dict['C06793_c'] = C06793_c
sk_dict['C06547_c'] = C06547_c
sk_dict['C07083_c'] = C07083_c
sk_dict['C06337_c'] = C06337_c

sk_dict['C11505_e'] = C11505_e
sk_dict['C06793_e'] = C06793_e
sk_dict['C06547_e'] = C06547_e
sk_dict['C07083_e'] = C07083_e
sk_dict['C06337_e'] = C06337_e

# prepare and run FBA

In [14]:
# create algorithm that takes a dataset, a model and a list of plastic metabolites and runs FBA 

def fba(df, model, plastics):
    
    # get pathways from data frame
    pathways = [i for i in df['INTERMEDIATES KEGG']]
    
    

    # change name of plastics so that it matches version in dict
    for i in pathways:
        for e, compound in enumerate(i):
            if compound in plastics:
                i[e] = compound + '_c'
                
    
    
    # get list of cytosolic plastics (for later analysis)
    plastics_c = plastics.copy()

    for e, compound in enumerate(plastics_c):
        plastics_c[e] = compound + '_c'
     
    
            
    # get external plastics        
    e_plastics = plastics.copy()
    
    for i, plastic in enumerate(plastics):
        e_plastics[i] = plastic + '_e'
   
     
    
    # get list of products    
    products = [i[-1] for i in pathways]
    products = list(set(products))
    
    
    
    # get external products
    e_products = products.copy()
    
    for i, prod in enumerate(products):
        e_products[i] = prod + '_e'

    
    
    # get name of compounds that are not present in model
    get_compound = set()

    for pathway in pathways:
        for compound in pathway:
            if compound not in plastic_dict and compound not in sk_dict.values() and compound not in get_compound:
                get_compound.add(compound)

    get_compound = list(get_compound)
    
    

    # create names for missing metabolites
    compound_name = []

    for i, compound in enumerate(get_compound):
        comp = f"Metabolite_{i}"
        compound_name.append(comp)

        
    
    # take list of missing compounds, create metabolites and add to dictionaries
    metabolites = []
    met_dict = {}

    for compound in get_compound:
        new_met = Metabolite(compound, compartment='c')
        metabolites.append(new_met)
        sk_dict[compound] = new_met
        met_dict[compound] = new_met
        
    
    
    # add external plastics to list of pathways
    ext = []
    
    for pathway in pathways:
        if pathway[0] in plastics_c:
            index = plastics_c.index(pathway[0])
            ext.append(e_plastics[index])
        
        
    
    # add external products (weren't in list before) and add to model
    ext_p = []

    for i in e_products:
        new_prod = Metabolite(i, compartment='e')
        ext_p.append(new_prod)
        sk_dict[i] = new_prod

    model.add_metabolites(ext_p)
    
    
    # add external products to list of pathways
    ext2 = []
    
    for pathway in pathways:
        if pathway[-1] in products:
            index = products.index(pathway[-1])
            ext2.append(e_products[index])

    
            
    # add external lists to pathways     
    for i, item in enumerate(ext):
        pathways[i].insert(0, item)
    
    for j, item in enumerate(ext2):
        pathways[j].append(item)

        
        
    # transform compounds in pathways list to metabolites
    for i in pathways:
        for j, item in enumerate(i):
            
            found = False
            

            if item in sk_dict:
                i[j] = sk_dict[item]
                found = True

            if not found and item in met_dict:
                i[j] = sk_dict[item]
                found = True

            if not found and item in sk_dict.values():
                temp = [key for key, value in sk_dict.items() if value == item]
                if temp:
                    temp2 = temp[0]
                    met = model.metabolites.get_by_id(temp2)
                    i[j] = met
                    found = True

            if not found:
                i[j] = item


    
    # separate list of pathways into twos (metabolite_1 --> metabolite 2)
                                        # (metabolite_2 --> metabolite_3)
                                        #  etc. 
    separated_pathways = []

    for i in pathways:
        pathway_pairs = []
        for j in range(len(i)-1):
            pair = [i[j], i[j+1]]
            pathway_pairs.append(pair)
        separated_pathways.append(pathway_pairs)
        
    
    
    # get number of reactions in pathway
    react_num = 0
    for i in separated_pathways:
        react_num += len(i)
        
        
     
    # create empty reactions using number of reactions in pathways
    reactions = []

    for i in range(react_num):
        reaction = f"Reaction_{i}"

        new_reaction = Reaction(reaction)

        reactions.append(new_reaction)
        
        
        
    # create output list    
    grows = []
   

    
    # add metabolites to reactions and reaction to model
    for m, e in enumerate(separated_pathways):
        
        reactions_added = []
        
        for i, item in enumerate(e):
            
            # Add reaction to the model
            model.add_reactions([reactions[i]])
            
            # Add metabolites to reaction
            reactions[i].add_metabolites({
                    item[0]: -1.0,
                    item[1]: 1.0
                    })


            # for first and last reactions, create exchange reactions
            if i == 0 or i == len(e)-1:
                reactions[i].lower_bound = -10.0
                reactions[i].upper_bound = 1000.0
                
                
            # for all others, create normal (oneway) reactions
            else:
                reactions[i].lower_bound = 0.0
                reactions[i].upper_bound = 1000.0
 

            reactions_added.append(reactions[i])
            
            
            
        # run FBA on model    
        solution = model.optimize()
        
        
        
        # if cell can grow, store results
        if solution.status == 'optimal' and solution.objective_value > 0.000:

            grows.append([m, solution])

            # remove all reactions and metabolites added within the loop
            model.remove_reactions(reactions_added)

            reactions = [reaction for reaction in reactions if reaction not in reactions_added]

            del reactions_added[:]

            

        else:
            
            # remove all reactions and metabolites added within the loop
            model.remove_reactions(reactions_added)

            reactions = [reaction for reaction in reactions if reaction not in reactions_added]

            del reactions_added[:]
            
    
    
    return (grows)

In [15]:
grows = fba(df, model, plastics)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [16]:
grows

[[25, <Solution 9.379 at 0x7feb7e053510>],
 [29, <Solution 0.000 at 0x7feb7e0538d0>],
 [136, <Solution 9.379 at 0x7feb795a6a10>],
 [188, <Solution 9.379 at 0x7feb7b378450>],
 [254, <Solution 9.379 at 0x7feb7ba148d0>],
 [348, <Solution 0.000 at 0x7feb7913d950>],
 [378, <Solution 6.127 at 0x7feb78d06fd0>],
 [379, <Solution 6.127 at 0x7feb78d06650>],
 [380, <Solution 6.127 at 0x7feb78f70710>],
 [381, <Solution 6.323 at 0x7feb7db2ea10>],
 [382, <Solution 6.323 at 0x7feb7db2eb50>],
 [383, <Solution 6.127 at 0x7feb7db2e890>],
 [384, <Solution 6.127 at 0x7feb7da575d0>],
 [385, <Solution 6.127 at 0x7feb785ca150>],
 [386, <Solution 6.127 at 0x7feb79168bd0>],
 [387, <Solution 6.127 at 0x7feb7917ddd0>],
 [388, <Solution 6.127 at 0x7feb785b8c50>],
 [389, <Solution 6.323 at 0x7feb785b8250>],
 [390, <Solution 6.323 at 0x7feb785a6f50>],
 [391, <Solution 6.127 at 0x7feb785a6250>],
 [392, <Solution 6.127 at 0x7feb781889d0>],
 [393, <Solution 6.127 at 0x7feb78188390>],
 [394, <Solution 6.127 at 0x7feb78

# generate data frame from results

In [17]:
grows_df = pd.DataFrame()

grows_df['index'] = [i[0] for i in grows]

grows_df['value'] = [i[1].objective_value for i in grows]

grows_df

,index,value
0,25,9.379374e+00
1,29,1.186644e-17
2,136,9.379374e+00
3,188,9.379374e+00
4,254,9.379374e+00
5,348,2.138682e-17
6,378,6.126559e+00
7,379,6.126559e+00
8,380,6.126559e+00
9,381,6.322964e+00


## export

In [18]:
grows_df.to_csv('solution_final.csv', index = False)

# analyse solutions and generate final df

In [19]:
# get number of pathways that can produce growth
num_growing_pathways = len(grows)
num_growing_pathways

42

In [20]:
# get indices of growing pathways
ind = [i[0] for i in grows]

In [21]:
# get dataframe of growing pathways
new_df = csv.iloc[ind]
new_df

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES,index
25,6,"[C06793_e, C06793_c, C20609, s_0445, s_4184, C...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",54
29,6,"[C06793_e, C06793_c, s_3976, s_3975, C01471, C...","[Vinyl chloride, trans-3-Chloroacrylic acid, t...",58
136,6,"[C06793_e, C06793_c, s_3976, s_0445, s_4184, C...","[Vinyl chloride, trans-3-Chloroacrylic acid, H...",169
188,6,"[C06793_e, C06793_c, C20609, s_0445, s_4184, C...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",228
254,6,"[C06793_e, C06793_c, s_3976, s_0445, s_4184, C...","[Vinyl chloride, trans-3-Chloroacrylic acid, H...",296
348,6,"[C06793_e, C06793_c, s_3976, s_3975, C01471, C...","[Vinyl chloride, trans-3-Chloroacrylic acid, t...",429
378,6,"[C06793_e, C06793_c, C06753, s_1275, C14448, s...","[Vinyl chloride, 2-Chloroethanol, Oxygen, Glyo...",465
379,6,"[C06793_e, C06793_c, C20303, s_1275, s_0721, s...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",466
380,6,"[C06793_e, C06793_c, C20303, s_1275, s_0721, s...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",467
381,6,"[C06793_e, C06793_c, C20303, s_1275, s_0779, s...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",468


In [22]:
# dataframe contains indeces of original data frame
indexog = [i for i in new_df['index']]

In [23]:
# generate final dataframe of growing 
final = ogcsv.iloc[indexog]
final

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES
54,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox..."
58,6,"[C06793, C06614, C06613, C01471, C05608, C0656...","[Vinyl chloride, trans-3-Chloroacrylic acid, t..."
169,6,"[C06793, C06614, C00288, C00222, C10020, C0460...","[Vinyl chloride, trans-3-Chloroacrylic acid, H..."
228,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox..."
296,6,"[C06793, C06614, C00288, C00222, C10020, C0460...","[Vinyl chloride, trans-3-Chloroacrylic acid, H..."
429,6,"[C06793, C06614, C06613, C01471, C00903, C1640...","[Vinyl chloride, trans-3-Chloroacrylic acid, t..."
465,6,"[C06793, C06753, C00007, C14448, C00546, C1972...","[Vinyl chloride, 2-Chloroethanol, Oxygen, Glyo..."
466,6,"[C06793, C20303, C00007, C00067, C00546, C1972...","[Vinyl chloride, Chloroethylene oxide, Oxygen,..."
467,6,"[C06793, C20303, C00007, C00067, C00546, C1228...","[Vinyl chloride, Chloroethylene oxide, Oxygen,..."
468,6,"[C06793, C20303, C00007, C00048, C00546, C1972...","[Vinyl chloride, Chloroethylene oxide, Oxygen,..."


In [24]:
# get names of source-targets
pairs = [i[0]+' - '+ i[-1] for i in final['INTERMEDIATE NAMES']]
pairs

['Vinyl chloride - Naringenin',
 'Vinyl chloride - Naringenin',
 'Vinyl chloride - Naringenin',
 'Vinyl chloride - Apigenin',
 'Vinyl chloride - Apigenin',
 'Vinyl chloride - Pinocembrin',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Vinyl chloride - beta-Carotene',
 'Viny

In [25]:
# add column containing values of FBA
solution_values = [i[1].objective_value for i in grows]
final = final.assign(solution=solution_values)
final

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES,solution
54,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",9.379374e+00
58,6,"[C06793, C06614, C06613, C01471, C05608, C0656...","[Vinyl chloride, trans-3-Chloroacrylic acid, t...",1.186644e-17
169,6,"[C06793, C06614, C00288, C00222, C10020, C0460...","[Vinyl chloride, trans-3-Chloroacrylic acid, H...",9.379374e+00
228,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",9.379374e+00
296,6,"[C06793, C06614, C00288, C00222, C10020, C0460...","[Vinyl chloride, trans-3-Chloroacrylic acid, H...",9.379374e+00
429,6,"[C06793, C06614, C06613, C01471, C00903, C1640...","[Vinyl chloride, trans-3-Chloroacrylic acid, t...",2.138682e-17
465,6,"[C06793, C06753, C00007, C14448, C00546, C1972...","[Vinyl chloride, 2-Chloroethanol, Oxygen, Glyo...",6.126559e+00
466,6,"[C06793, C20303, C00007, C00067, C00546, C1972...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",6.126559e+00
467,6,"[C06793, C20303, C00007, C00067, C00546, C1228...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",6.126559e+00
468,6,"[C06793, C20303, C00007, C00048, C00546, C1972...","[Vinyl chloride, Chloroethylene oxide, Oxygen,...",6.322964e+00


## export

In [27]:
final.to_csv('final.csv', index = False)

# get max pathway

In [28]:
max_solution_position, max_solution = max(enumerate(grows), key=lambda x: x[1][1].objective_value)
max_index = max_solution[0]
max_obj_value = max_solution[1].objective_value

print("Maximum objective value:", max_obj_value)
print("Position of the row with maximum objective value:", max_solution_position)

Maximum objective value: 9.37937412001673
Position of the row with maximum objective value: 3


In [29]:
max_pathway = pd.DataFrame(final.iloc[max_solution_position:max_solution_position+1])
max_pathway

,LENGTH,INTERMEDIATES KEGG,INTERMEDIATE NAMES,solution
228,6,"[C06793, C20609, C00288, C00222, C10020, C0460...","[Vinyl chloride, 2-Chloroacrylate, H2CO3, 3-Ox...",9.379374


## export

In [30]:
max_pathway.to_csv('max_pathway.csv', index = False)